# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298713842688                   -0.85    5.8    216ms
  2   -8.300236354105       -2.82       -1.26    1.0   98.6ms
  3   -8.300444264495       -3.68       -1.90    3.6    160ms
  4   -8.300461357753       -4.77       -2.78    1.0   96.6ms
  5   -8.300463939267       -5.59       -3.03    3.0    132ms
  6   -8.300464366779       -6.37       -3.24    1.1    104ms
  7   -8.300464518234       -6.82       -3.39    1.0   92.0ms
  8   -8.300464593131       -7.13       -3.55    1.0   91.8ms
  9   -8.300464632683       -7.40       -3.76    1.2   95.7ms
 10   -8.300464634978       -8.64       -3.81    1.0   94.0ms
 11   -8.300464642841       -8.10       -4.11    1.0   92.7ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67565683242                   -0.70    7.5    526ms
  2   -16.67880195027       -2.50       -1.14    1.0    209ms
  3   -16.67922589307       -3.37       -1.86    2.0    245ms
  4   -16.67927302198       -4.33       -2.74    1.0    206ms
  5   -16.67928561687       -4.90       -3.19    5.9    385ms
  6   -16.67928614379       -6.28       -3.48    3.6    287ms
  7   -16.67928621204       -7.17       -3.89    1.8    225ms
  8   -16.67928621973       -8.11       -4.49    2.2    246ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32646459315                   -0.56    7.9    1.46s
  2   -33.33290946818       -2.19       -1.00    1.0    692ms
  3   -33.33414239551       -2.91       -1.73    4.1    932ms
  4   -33.33426521758       -3.91       -2.64    1.2    685ms
  5   -33.33519105305       -3.03       -2.43    5.2    1.20s
  6   -33.33694293836       -2.76       -2.52    7.8    1.17s
  7   -33.33694304449       -6.97       -2.52    1.0    622ms
  8   -33.33685855774   +   -4.07       -2.39    2.4    668ms
  9   -33.33673267828   +   -3.90       -2.21    2.1    652ms
 10   -33.33671922215   +   -4.87       -2.20    1.0    617ms
 11   -33.33673744542       -4.74       -2.21    1.0    617ms
 12   -33.33675787032       -4.69      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298573564907                   -0.85    5.2    257ms
  2   -8.300269908682       -2.77       -1.59    1.1   81.2ms
  3   -8.300429063398       -3.80       -2.59    3.2    104ms
  4   -8.300407130762   +   -4.66       -2.37    4.6    162ms
  5   -8.300463675541       -4.25       -3.24    1.0   87.3ms
  6   -8.300464571691       -6.05       -3.67    3.6    141ms
  7   -8.300464637246       -7.18       -4.22    1.4   98.5ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32608097885                   -0.56    6.9    1.70s
  2   -33.30723333006   +   -1.72       -1.26    1.0    579ms
  3   -8.443423133991   +    1.40       -0.37    6.6    1.49s
  4   -33.32967251917        1.40       -1.82    6.1    1.36s
  5   -33.18815777352   +   -0.85       -1.28    3.9    1.07s
  6   -32.68122257707   +   -0.30       -1.15    3.9    1.06s
  7   -33.32235509496       -0.19       -1.95    4.9    1.04s
  8   -33.33455554383       -1.91       -2.23    2.4    715ms
  9   -33.33608394639       -2.82       -2.34    2.5    834ms
 10   -33.33683006188       -3.13       -2.89    1.8    697ms
 11   -33.33687717778       -4.33       -3.11    3.4    854ms
 12   -33.33693759062       -4.22      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.